In [1]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm

from torch.utils.data import Dataset,DataLoader

In [2]:
'''
1， 迁移学习是认为别人的Resnet，googlenet这些输出的是特征，是图像的特征。我们自己建立分类模型，对着1000个特征进行处理即可
2， finetune是把别人的模型的已经训练好的参数，作为我们的初始化参数，这样，收敛速度快，而且需要的计算力也小。
'''

'\n1， 迁移学习是认为别人的Resnet，googlenet这些输出的是特征，是图像的特征。我们自己建立分类模型，对着1000个特征进行处理即可\n2， finetune是把别人的模型的已经训练好的参数，作为我们的初始化参数，这样，收敛速度快，而且需要的计算力也小。\n'

In [3]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [4]:
from utils.dataloader_image_classification import ImageTransform,make_datapath_list,HymenopteraDataSet

size = 224
mean = [0.485,0.456,0.406]
std = [0.229,0.224,0.225]

train_list = make_datapath_list(phase='train')
val_list = make_datapath_list(phase='val')

train_dataset = HymenopteraDataSet(file_list=train_list,transform=ImageTransform(size,mean,std),phase='train')
val_dataset = HymenopteraDataSet(file_list=val_list,transform=ImageTransform(size,mean,std),phase='val')   

#  创建DataLoader
batch_size = 32

train_dataloader = DataLoader(train_dataset,batch_size,shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size,shuffle=False)

# 集中到字典变量中
dataloader_dict = {'train':train_dataloader,'val':val_dataloader}

In [5]:
# 创建网络模型
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)

net.classifier[6] = nn.Linear(in_features=4096,out_features=2)

net.train()

# 损失函数
criterion = nn.CrossEntropyLoss()

In [6]:
# 微调算法与迁移学习 不同， 这里将optimizer设置为全部网络层参数都可以学习

# python 列表是可变类型，里面存储的元素会自动更新最新值

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

for name, param in net.named_parameters():
    
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("保存到params_to_update_1：", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("保存到params_to_update_2：", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("保存到params_to_update_3：", name)

    else:
        param.requires_grad = False
        print("不计算梯度，不学习", name)


保存到params_to_update_1： features.0.weight
保存到params_to_update_1： features.0.bias
保存到params_to_update_1： features.2.weight
保存到params_to_update_1： features.2.bias
保存到params_to_update_1： features.5.weight
保存到params_to_update_1： features.5.bias
保存到params_to_update_1： features.7.weight
保存到params_to_update_1： features.7.bias
保存到params_to_update_1： features.10.weight
保存到params_to_update_1： features.10.bias
保存到params_to_update_1： features.12.weight
保存到params_to_update_1： features.12.bias
保存到params_to_update_1： features.14.weight
保存到params_to_update_1： features.14.bias
保存到params_to_update_1： features.17.weight
保存到params_to_update_1： features.17.bias
保存到params_to_update_1： features.19.weight
保存到params_to_update_1： features.19.bias
保存到params_to_update_1： features.21.weight
保存到params_to_update_1： features.21.bias
保存到params_to_update_1： features.24.weight
保存到params_to_update_1： features.24.bias
保存到params_to_update_1： features.26.weight
保存到params_to_update_1： features.26.bias
保存到params_to_update_1： f

In [7]:
# 设置各参数的最优化算法
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

In [8]:
def train_model(net,dataloader_dict,criterion,optimizer,num_epochs):
    
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    net.to(device)

    # 如果网络达到稳定的程度，则开启加速
    torch.backends.cudnn.benchmark = False

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1,num_epochs))
        print('----'*5)

        # 每个epoch中的学习和循环
        for phase in ['train','val']:
            if phase=='train':
                net.train()
            else:
                net.eval()

            epoch_loss = 0.0
            epoch_corrects = 0

            # 为了对比，未训练与训练的验证能力，epoch=0时候，不训练，直接进行val
            if (epoch == 0) and (phase == 'train'):
                continue

            # 载入数据
            for inputs,labels in tqdm(dataloader_dict[phase]):

                inputs = inputs.to(device)
                labels = labels.to(device)

                # 初始化optimizer
                optimizer.zero_grad()

                # 计算正向传播
                with torch.set_grad_enabled(phase=='train'):
                    outputs = net(inputs)
                    loss = criterion(outputs,labels)
                    _,preds = torch.max(outputs,1)

                    # 训练状态 进行反向传播
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 计算迭代结果
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)

            # 显示每个epoch的loss和准确率
            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
            
            print('{} loss: {:.4f} Acc:{:.4f}'.format(phase, epoch_loss,epoch_acc))

        print('============'*5)

In [9]:
# 运行
num_epochs = 2
train_model(net,dataloader_dict,criterion,optimizer,num_epochs=num_epochs)

Epoch 1/2
--------------------


100%|██████████| 5/5 [00:21<00:00,  4.30s/it]


val loss: 0.7703 Acc:0.4444
Epoch 2/2
--------------------


100%|██████████| 8/8 [00:08<00:00,  1.10s/it]


train loss: 0.5051 Acc:0.7325


100%|██████████| 5/5 [00:01<00:00,  3.26it/s]

val loss: 0.1829 Acc:0.9608


In [10]:
# 保存模型
save_path = './model_params/vgg16_fine_tuning.pth'
torch.save(net.state_dict(),save_path)

In [11]:
# 载入模型
load_path = './model_params/vgg16_fine_tuning.pth'
load_weights = torch.load(load_path)

net = models.vgg16(pretrained=use_pretrained)
net.classifier[6] = nn.Linear(in_features=4096,out_features=2)
net.load_state_dict(load_weights)  # GPU模型

'''
# 在GPU上保存的权重，使用CPU读取
'''
net = models.vgg16(pretrained=use_pretrained)
net.classifier[6] = nn.Linear(in_features=4096,out_features=2)
net.load_state_dict(load_weights)
load_weights = torch.load(load_path,map_location={'cuda:0':'cpu'})
net.load_state_dict(load_weights) # CPU模型

<All keys matched successfully>